### RNN 활용하기

simpleRNN, LSTM, GRU의 구조를 간단하게 알고, 예제를 통해 순환 신경망을 응용해본다.

### Embedding
컴퓨터는 우리가 사용하는 한글, 영어 등을 그대로 이해하지 못한다. 컴퓨터가 이해하는 벡터로 변경해야 하는데, 그 과정을 `Embedding`이라고 한다. 임베딩이 잘 될수록 모델의 성능 또한 높아지도 훈련도 잘되기 때문에 임베딩과 관련된 여러가지 방법들이 개발되어 있다.

크게는 단어 수준의 임베딩(Word2Vec, FastText)과 문장 수준의 임베딩(Elmo, Bert, GPT)으로 구분할 수 있다. 단어 수준의 임베딩을 위해 단어, 음절, 형태소 등으로 나누고 이를 수치로 변환한다. 가장 간단한 벡터 변환 방법은 원핫 인코딩이다.

하지만 원핫 인코딩의 경우 단어가 많아질수록 벡터의 공간이 커지고 비효율성이 발생하는 문제가 있다. 원핫 인코딩은 행렬 값이 대부분 0인 희소행렬이라고도 부른다. 엄청난 크기의 희소행렬은 공간복잡도 부분에서 불리하고, 원핫 인코딩은 각 단어 간의 유사도를 알기 어렵다는 한계가 있다.

Tensorflow에서는 임베딩 레이어를 제공하고 있는데, 희소행렬이 아닌 밀집행렬(0이 아닌 실수값으로 채워진 행렬)로 구현되어 있다. 이는 희소행렬과는 달리 단어간의 유사도도 알 수 있다.

In [5]:
import tensorflow as tf

# 임베딩 레이어
embed_layer = tf.keras.layers.Embedding(100, 3)
result = embed_layer(tf.constant([12, 8, 15, 20]))
result.shape

TensorShape([4, 3])

임베딩 레이어는 input_dim, output_dim, input_length 파라미터가 있다. input_dim은 사전의 크기를 의미한다. output_dim은 출력 벡터의 크기를 말한다.

임베딩 레이어를 모델에 적용한다. 일반적으로 자연어 처리 모델에서는 데이터를 입력하는 부분에 임베딩 레이어를 활용한다. 또한 모델에 따라 입력 길이를 설정해야 할 수도 있다.

In [6]:
# 입력 길이를 32로 설정
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(100, 3, input_length=32))
model.add(tf.keras.layers.LSTM(units = 32))
model.add(tf.keras.layers.Dense(units = 1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 32, 3)             300       
                                                                 
 lstm (LSTM)                 (None, 32)                4608      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 4,941
Trainable params: 4,941
Non-trainable params: 0
_________________________________________________________________


### 양방향(Bidirectional) RNN
자연어 데이터의 경우 순서대로 데이터를 처리하고, 역순으로 처리할 경우 더 좋은 성능을 발휘하기도 한다. 

![screensh](../screenshots/bidirectional%20RNN.png)

만약 아래와 같은 문장을 예측한다고 가정하자.

> 나는 _____ 를 보고싶다.

이를 양방향으로 예측을 진행하면 영화, 친구, 엄마, 여자친구 같은 단어들을 생각할 수 있다. 앞에 있는 `나는`이라는 단어를 토대로 예측할 수 있는 결과는 많지 않지만, 뒤에 나오는 `보고싶다`라는 단어를 기반으로 예측할 수 있는 개수는 많기 때문이다.

In [1]:
# Bidirectional LSTM
from keras.layers import Bidirectional, Embedding, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(Embedding(100, 3))
model.add(Bidirectional(LSTM(32))) # 양방향 RNN. 32개의 유닛 개수를 지정했지만 양방향이기에 계층도 내부에서는 64개로 보인다.
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 3)           300       
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9216      
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 9,581
Trainable params: 9,581
Non-trainable params: 0
_________________________________________________________________


### Stacking RNN
1 ~ t까지의 시점이 있다고 가정하자. 일반적인 RNN의 경우에는 1 ~ t-1 시점까지의 출력값은 모두 무시한다. 마지막 출력값만을 가지고 예측을 진행하는데, 이러한 구조로는 RNN층을 쌓을 수 없다. RNN의 파라미터인 `return_sequences`를 True로 설정하게 되면, 1 ~ t-1 시점까지의 출력값도 모두 반영하게 되며, 이 때 비로소 `Stacking RNN`을 구현할 수 있다. 만약 `return_sequences`를 기본값인 False로 방치하게 되면 오류가 발생하며 모델을 빌드할 수 없게 될 것이다. 최하단에서 다음 층으로 넘어갈 때 전달되는 형태는 (batch_size, timesteps, units)가 전달된다. 

이러한 구조에 따라, 최상단에 위치하는 RNN의 경우에는 return_sequences를 True로 설정할 필요가 없다.

In [2]:
model = Sequential()
model.add(Embedding(100, 32))
model.add(LSTM(32, return_sequences=True)) # return_sequences를 True로 설정해 다음 RNN 층에 모든 출력 결과를 넘긴다. (batch_size, timesteps, units)
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          3200      
                                                                 
 lstm_1 (LSTM)               (None, None, 32)          8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 19,873
Trainable params: 19,873
Non-trainable params: 0
_________________________________________________________________


### Recurrent Dropout
keras에서 일반적으로 제공되는 Dropout 계층과는 약간 다르다. 일반적인 Dropout의 경우 Input값에 대한 dropout인 반면, Recurrent Dropout의 경우 `순환상태`에 대한 dropout 비율을 이야기한다. 모두 0과 1사이의 부동소수점이고 과대적합을 방지하기 위해 사용한다.

In [4]:
import warnings

warnings.filterwarnings("ignore")

model = Sequential()
model.add(Embedding(100, 32))
model.add(LSTM(32, recurrent_dropout = 0.2, dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          3200      
                                                                 
 lstm_4 (LSTM)               (None, 32)                8320      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 11,553
Trainable params: 11,553
Non-trainable params: 0
_________________________________________________________________
